In [1]:
import matplotlib
#%matplotlib tk
%matplotlib inline
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.cohort import cohort
from utils.ethogram import ethogram

Autosaving every 180 seconds


In [2]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
#fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = cohort.CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

#
cohort.ethogram = ethogram

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0           0       2     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1           1       2     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2           2       2     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3           3       2     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4           4       

In [3]:
#################################################
########### COMPUTE HUDDLE COMPOSITION ##########
#################################################

#
cohort.video_frame_rate = 24
cohort.parallel_flag = True

# set the flags for the huddle corrections
cohort.fix_track_flag = True
cohort.interpolate_flag = True
cohort.load_huddle_tracks()

# # this just loads the _spine.npy files;
cohort.use_nohuddle = True
cohort.load_feature_tracks()

# # this computes mode-based occupancy in the nest with 1min bins
cohort.median_filter_width = cohort.video_frame_rate*5+1 # 1: there is no smotthing; n x frame rate gives you n seconds of somothing (always add 1 to get odd number)
#cohort.n_frames_per_bin = cohort.video_frame_rate  # single second resolution
cohort.n_frames_per_bin = cohort.video_frame_rate #*60  # single second resolution
cohort.n_cores = 24
cohort.compute_huddle_composition()
print ("# of huddle composition arrays: ", len(cohort.huddle_comps_binned), cohort.huddle_comps_binned[0].shape)


 # of files:  991 example:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/huddles/2020_07_20_09_53_59_813912_compressed_Day_huddle_spine_fixed_interpolated.npy
# of feature tracks:  991


  0%|          | 0/991 [00:00<?, ?it/s]

# of huddle composition arrays:  991 (6, 1201)


In [6]:
#################################################
########### DISPLAY HUDDLE COMPOSITION ##########
#################################################

#
cohort.forward_padding = 20 # padding in percent
cohort.generate_huddle_composition_ethograms()

#
#cohort.show_huddle_composition_ethogram_all_animals()

#
#cohort.show_huddle_composition_ethogram_full_huddle()

#
animal_ids = np.arange(6)
cohort.show_huddle_composition_ethogram_specific_animals(animal_ids)


size of img:  (86400, 96)  size of flatten image:  (1382400, 6)


100%|███████████████████████████████████████| 991/991 [00:00<00:00, 2564.01it/s]


In [104]:
##########################################################
########### COMPUTE FLATTENED VERSION OF ETHOGRAM ########
##########################################################

# OPTIONAL
# print (cohort.huddle_ethogram.shape)

# # flatten the ethogram
# ethogram_flat = []

# for k in range(0,cohort.huddle_ethogram.shape[1],6):
#     ethogram_flat.append(cohort.huddle_ethogram[:,k:k+6])
# ethogram_flat = np.vstack(ethogram_flat).T
# print (ethogram_flat.shape)

# # optional visualize
# if False:
#     plt.figure()

#     plt.imshow(ethogram_flat, aspect='auto', 
#                interpolation='none',  cmap='jet')
#     plt.show()

(86400, 96)
(6, 1382400)


In [7]:
##############################################################################
########### COMPUTE FLATTENED VERSION OF ETHOGRAM + CORRELATION ARRAY ########
##############################################################################

# FIGURE 1: CORRELOGRAMS <- not super informative!
# flatten the ethogram
ethogram_flat = []

for k in range(0,cohort.huddle_ethogram.shape[1],6):
    ethogram_flat.append(cohort.huddle_ethogram[:,k:k+6])
ethogram_flat = np.vstack(ethogram_flat).T
print (ethogram_flat.shape)

# compute the correlograms between each pair of animals
from scipy.stats import pearsonr

#
window = 100
t = np.arange(-window,window,1)
from tqdm import trange 

#
cc_array = []
plt.figure()
for k in range(6):
    for p in trange(k+1,6,1):
        cc = []
        for z in range(-window, window,1):
            cc.append(pearsonr(np.roll(ethogram_flat[k],z), 
                                    ethogram_flat[p])[0])
        cc_array.append([k,p,cc])

        #
        ax=plt.subplot(6,6,k*6+p+1)
        plt.plot(t,cc)
        plt.plot([0,0],[-1,1],'--',c='black')
#

plt.show()

(6, 1382400)


100%|█████████████████████████████████████████████| 1/1 [00:18<00:00, 18.99s/it]
0it [00:00, ?it/s]


In [115]:
# cc = []
# k = 0
# p = 1
# window = 1000
# t=np.arange(-window,window,1)
# t1 = ethogram_flat[k][520000:630000].copy()
# idx = np.where(t1>0)
# t1[idx] = 1
# diffs = t1[1:]-t1[:-1]
# idx1 = np.where(diffs>0)
# idx2 = np.where(diffs<0)
# t1[:] = 0
# t1[idx1] = 1
# t1[idx2] = -1

# #
# t2 = ethogram_flat[p][520000:630000].copy()
# idx = np.where(t2>0)
# t2[idx] = 1
# diffs = t2[1:]-t2[:-1]
# idx1 = np.where(diffs>0)
# idx2 = np.where(diffs<0)
# t2[:] = 0
# t2[idx1] = 1
# t2[idx2] = -1

# #
# for z in trange(-window, window,1):
#     cc.append(pearsonr(np.roll(t1,z), 
#                         t2)[0])

# #
# #plt.plot([0,0],[0,1],'--',c='black')
# plt.xlabel("seconds")
# #plt.plot([0,0],[50,50],'--',c='black')
# plt.plot(t,cc)
# plt.show()

100%|██████████| 2000/2000 [00:10<00:00, 182.76it/s]


In [106]:
####################################################
########### COMPUTE PAIR WISE SCATTER PLOTS ########
####################################################

# # flatten the ethogram
# ethogram_flat = []

# for k in range(0,cohort.huddle_ethogram.shape[1],6):
#     ethogram_flat.append(cohort.huddle_ethogram[:,k:k+6])
# ethogram_flat = np.vstack(ethogram_flat).T
# print (ethogram_flat.shape)

# plt.figure(figsize=(6,6))
# clrs = ['red','blue','green','black','purple','orange']
# markers = ["," , "o" , "v" , "^" , "<", ">"]

# #
# for k in trange(6):
#     for p in range(k+1,6,1):
#         t1 = ethogram_flat[k].copy()
#         idx = np.where(t1>0)
#         t1[idx] = 1
#         t2 = ethogram_flat[p].copy()
#         idx = np.where(t2>0)
#         t2[idx] = 1

#         # find order of events
#         diff = t1-t2
#         diff2 = diff[1:] - diff[:-1]

#         #
#         idx1 = np.where(diff2==2)[0]
#         idx2 = np.where(diff2==-2)[0]
#         plt.scatter(idx1.shape[0],
#                     idx2.shape[0],
#                     marker=markers[k],
#                     s=100,
#                     c=clrs[p],
#                     label=str(k)+'->'+str(p))

# #
# plt.plot([0,70],[0,70],'--',c='black')
# plt.legend()
# plt.title("# of behavior sequences (1sec resolution))")
# plt.xlim(left=0)
# plt.ylim(bottom=0)
# plt.xlabel("Animal 1 precedes animal 2")
# plt.ylabel("Animal 2 precedes animal 1")
# plt.show()



100%|██████████| 6/6 [00:00<00:00, 12.49it/s]


In [15]:
####################################################
########### SELECT DAY OF ETHOGRAM TO ANALYZ #######
####################################################

print ("# animals, # seconds: ", ethogram_flat.shape)

# here P15 is day 0 for clarity
day_start = 0
day_end = 1

#
times = np.arange(day_start*24*60*60,
                  day_end*24*60*60)
print ("times: ", times)

#
ethogram_flat_selected = ethogram_flat[:,times].copy()
print (ethogram_flat_selected.shape)




# animals, # seconds:  (6, 1382400)
times:  [    0     1     2 ... 86397 86398 86399]
(6, 86400)


In [21]:
####################################################
########### COMPUTE PAIR WISE SCATTER PLOTS ########
####################################################

plt.figure(figsize=(6,6))
clrs = ['red','blue','green','black','purple','orange']
markers = ["," , "o" , "v" , "^" , "<", ">"]

#
follow_window = 10
xlim=0
for k in trange(6):
    #for p in range(k+1,6,1):
    for p in range(6):
        t1 = ethogram_flat_selected[k].copy()
        idx1 = np.where(t1>0)[0]
        t1[idx1] = 1
        t2 = ethogram_flat_selected[p].copy()
        idx2 = np.where(t2>0)[0]
        t2[idx2] = 1

        # find order of events
        a1 = 0
        
        #
        for i in idx1:
            
            # check to see if there is a corresponding event in the other animal
            if t2[i]==0:
                # check if t2 is 1 at all for next 10 seconds
                if np.sum(t2[i:i+follow_window])>0:
                    a1+=1
        
        a2 = 0
        for i in idx2:
            
            # check to see if there is a corresponding event in the other animal
            if t1[i]==0:
                # check if t1 is 1 at all for next 10 seconds
                if np.sum(t1[i:i+follow_window])>0:
                    a2+=1

        xlim = np.max((xlim,a1,a2))
        plt.scatter(a1,
                    a2,
                    marker=markers[k],
                    s=100,
                    c=clrs[p],
                    label=str(k)+'->'+str(p))

#
plt.plot([0,xlim*1.25],[0,xlim*1.25],'--',c='black')
plt.legend(fontsize=8, ncol=6)
plt.title("# of behavior sequences (1sec resulution; 10sec follow window))")
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.title("start day: "+str(day_start) + ",  day end: "+str(day_end))
plt.xlabel("Animal 1 precedes animal 2")
plt.ylabel("Animal 2 precedes animal 1")
plt.show()


100%|█████████████████████████████████████████████| 6/6 [00:08<00:00,  1.34s/it]
